In [1]:
from __future__ import division, print_function
from gensim import models
from keras.callbacks import ModelCheckpoint
from keras.layers import Dense, Dropout, Reshape, Flatten, concatenate, Input, Conv1D, GlobalMaxPooling1D, Embedding
from keras.layers.recurrent import LSTM
from keras.models import Sequential
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Model
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd
import os
import collections
import re
import string

Using TensorFlow backend.


In [2]:
data = pd.read_csv('Resources/LabelledData.csv', header = None, delimiter='\t')

In [3]:
data.columns = ['Question', 'Label']

In [4]:
data.head()

,Question,Label
0,how did serfdom develop in and then leave russia,unknown
1,what films featured the character popeye doyle,what
2,how can i find a list of celebrities ' real names,unknown
3,what fowl grabs the spotlight after the chines...,what
4,what is the full form of .com,what


In [5]:
data.Label.unique()

array(['unknown', 'what', 'when', 'who', 'affirmation'], dtype=object)

In [6]:
data.shape

(1483, 2)

In [7]:
def clean_str(string):
    """
    Tokenization/string cleaning for all datasets
    """
    string = re.sub(r"\'s", " \'s", string)
    string = re.sub(r"\'ve", " \'ve", string)
    string = re.sub(r"n\'t", " n\'t", string)
    string = re.sub(r"\'re", " \'re", string)
    string = re.sub(r"\'d", " \'d", string)
    string = re.sub(r"\'ll", " \'ll", string)
    string = re.sub(r",", " , ", string)
    string = re.sub(r"!", " ! ", string)
    string = re.sub(r"\(", " \( ", string)
    string = re.sub(r"\)", " \) ", string)
    string = re.sub(r"\?", " \? ", string)
    string = re.sub(r"\s{2,}", " ", string)
    return string.strip().lower()

In [8]:

import csv
training = []
line_counter = 0
with open("Resources/LabelledData.csv",'r') as f:
    tsv = csv.reader(f, dialect='excel', delimiter='\t')
    for line in tsv:
        if line_counter < 900:
            training.append(line[0])
        else:
            break
            line_counter = line_counter+1
    # training_data = list(training)
    training_data = [s.strip() for s in training]
    # Split by words
    x_text = training_data
    x_text = [clean_str(sent) for sent in x_text]  

In [9]:

t = open('check.txt','wt')
label =[]
labels=[]
line_counter = 0
with open("Resources/LabelledData.csv",'r') as f:
    tsv = csv.reader(f, dialect='excel', delimiter='\t')
    for line in tsv:
        if line_counter<900:
            label.append(line[1])
        else:
            break
            line_counter =  line_counter +1
    labels =label
    tempList = []
    labelList = []
    for a in label:
        t.write(a)
        t.write('--')
        if not a in tempList:
            tempList.append(a)
            labelList.append(tempList.index(a))
            t.write(str(tempList.index(a)))
            t.write('\n')
        else:
            labelList.append(tempList.index(a))
    n_labels = len(labelList)
    target = []
    for x in range(0,n_labels):
        listFori = []
        i=0
        while i < len(tempList):
            if i == labelList[x]:
                listFori.append(1)
            else:
                listFori.append(0)
            i = i+1
        target.append(listFori)
    y = np.concatenate([target],0)

In [10]:
tempList

['unknown', 'what', 'when', 'who', 'affirmation']

In [11]:
print(len(x_text))

1483


In [12]:
x_text[1482]

'would this work on a 2008 ford edge with a naked roof'

In [13]:
MAX_WORDS = 10000
MAX_SEQUENCE_LENGTH = 1000
EMBEDDING_DIM = 100
# filter sizes of the different conv layers 
filter_sizes = [3,4,5]
num_filters = 512
embedding_dim = 100
# dropout probability
drop = 0.5
batch_size = 30
epochs =30

In [14]:
GLOVE_DIR='./GLOVE.6B/'
embeddings_index = {}
FILE=os.path.join(GLOVE_DIR, 'glove.6B.100d.txt')
f = open(FILE,encoding='utf-8')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()
print('Found %s word vectors.' % len(embeddings_index))

Found 400000 word vectors.


In [15]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
tokenizer  = Tokenizer(num_words = MAX_WORDS)
print(tokenizer.fit_on_texts(x_text))
sequences =  tokenizer.texts_to_sequences(x_text)
print(len(sequences))
word_index = tokenizer.word_index
print("unique words : {}".format(len(word_index)))
data = pad_sequences(sequences, maxlen=MAX_SEQUENCE_LENGTH)
labels = to_categorical(np.asarray(y))
print('Shape of data tensor:', data.shape)
print('Shape of label tensor:', y.shape)
print(len(labels))

None
1483
unique words : 3674
Shape of data tensor: (1483, 1000)
Shape of label tensor: (1483, 5)
1483


In [16]:
indices = np.arange(data.shape[0])
np.random.shuffle(indices)
data = data[indices]
y = y[indices]
nb_validation_samples = int(0.10 * data.shape[0])
x_train = data[:-nb_validation_samples]
y_train = y[:-nb_validation_samples]
x_test = data[-nb_validation_samples:]
y_test = y[-nb_validation_samples:]

In [23]:
print(len(x_train))

1335


In [24]:
print(len(x_test))

148


In [25]:
print(len(y_train))

1335


In [26]:
print(len(y_test))

148


In [27]:
embedding_matrix = np.zeros((len(word_index) + 1, EMBEDDING_DIM))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if embedding_vector is not None:
        # words not found in embedding index will be all-zeros.
        embedding_matrix[i] = embedding_vector

In [28]:
embedding_matrix.shape

(3675, 100)

In [29]:
from keras.layers import Embedding

embedding_layer = Embedding(len(word_index) + 1,
                            EMBEDDING_DIM,
                            weights=[embedding_matrix],
                               input_length=MAX_SEQUENCE_LENGTH,
                             trainable=False)

In [30]:
from keras.layers import Activation, Conv2D, Input, Embedding, Reshape, MaxPool2D, Concatenate, Flatten, Dropout, Dense, Conv1D
from keras.layers import MaxPool1D
from keras.models import Model
from keras.callbacks import ModelCheckpoint
from keras.optimizers import Adam
inputs = Input(shape=(MAX_SEQUENCE_LENGTH,), dtype='int32')
embedding = embedding_layer(inputs)

print(embedding.shape)
reshape = Reshape((MAX_SEQUENCE_LENGTH,EMBEDDING_DIM,1))(embedding)
#print(reshape.shape)

conv_0 = Conv2D(num_filters, kernel_size=(filter_sizes[0], embedding_dim), padding='valid', kernel_initializer='normal',activation='relu')(reshape)
conv_1 = Conv2D(num_filters, kernel_size=(filter_sizes[1], embedding_dim), padding='valid', kernel_initializer='normal',activation='relu')(reshape)
conv_2 = Conv2D(num_filters, kernel_size=(filter_sizes[2], embedding_dim), padding='valid', kernel_initializer='normal',activation='relu')(reshape)

maxpool_0 = MaxPool2D(pool_size=(MAX_SEQUENCE_LENGTH - filter_sizes[0] +1, 1), strides=(1,1), padding='valid')(conv_0)
maxpool_1 = MaxPool2D(pool_size=(MAX_SEQUENCE_LENGTH - filter_sizes[1] + 1, 1), strides=(1,1), padding='valid')(conv_1)
maxpool_2 = MaxPool2D(pool_size=(MAX_SEQUENCE_LENGTH - filter_sizes[2] + 1, 1), strides=(1,1), padding='valid')(conv_2)


concatenated_tensor = Concatenate(axis=1)([maxpool_0, maxpool_1, maxpool_2])
flatten = Flatten()(concatenated_tensor)
dropout = Dropout(drop)(flatten)
output = Dense(units=5, activation='softmax')(dropout)
print(output)

# this creates a model that includes
model = Model(inputs=inputs, outputs=output)

checkpoint = ModelCheckpoint('weights_cnn_sentece.hdf5', monitor='val_acc', verbose=1, save_best_only=True, mode='auto')
adam = Adam(lr=1e-4, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

model.compile(optimizer=adam, loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

(None, 1000, 100)
Tensor("dense_1/Softmax:0", shape=(None, 5), dtype=float32)
Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            (None, 1000)         0                                            
__________________________________________________________________________________________________
embedding_1 (Embedding)         (None, 1000, 100)    367500      input_1[0][0]                    
__________________________________________________________________________________________________
reshape_1 (Reshape)             (None, 1000, 100, 1) 0           embedding_1[0][0]                
__________________________________________________________________________________________________
conv2d_1 (Conv2D)               (None, 998, 1, 512)  154112      reshape_1[0][0]                  
______________

In [31]:
print("Traning Model...")
model.fit(x_train, y_train, batch_size=10, epochs=50 , verbose=1, callbacks=[checkpoint])

Traning Model...
Epoch 1/50
1335/1335 [==============================] - 119s 89ms/step - loss: 1.2963 - accuracy: 0.5356
Epoch 2/50


C:\Users\Chindukuri\.conda\envs\tensorflow\lib\site-packages\keras\callbacks\callbacks.py:707: RuntimeWarning: Can save best model only with val_acc available, skipping.
  'skipping.' % (self.monitor), RuntimeWarning)


1335/1335 [==============================] - 121s 91ms/step - loss: 0.7545 - accuracy: 0.7161
Epoch 3/50
1335/1335 [==============================] - 121s 91ms/step - loss: 0.5065 - accuracy: 0.8292
Epoch 4/50
1335/1335 [==============================] - 121s 90ms/step - loss: 0.3763 - accuracy: 0.8839
Epoch 5/50
1335/1335 [==============================] - 120s 90ms/step - loss: 0.2956 - accuracy: 0.9161
Epoch 6/50
1335/1335 [==============================] - 120s 90ms/step - loss: 0.2416 - accuracy: 0.9273
Epoch 7/50
1335/1335 [==============================] - 122s 91ms/step - loss: 0.1966 - accuracy: 0.9491
Epoch 8/50
1335/1335 [==============================] - 121s 90ms/step - loss: 0.1584 - accuracy: 0.9610
Epoch 9/50
1335/1335 [==============================] - 125s 93ms/step - loss: 0.1326 - accuracy: 0.9708
Epoch 10/50
1335/1335 [==============================] - 121s 91ms/step - loss: 0.1133 - accuracy: 0.9730
Epoch 11/50
1335/1335 [==============================] - 120s 90m

In [32]:
score = model.evaluate(x_test, y_test, verbose=0)  
print('Test loss:', score[0])
print('Test accuracy:', score[1])

Test loss: 0.08382667973637581
Test accuracy: 0.9729729890823364


In [33]:
predictions = model.predict(x_test, batch_size=1024,   verbose=1)
labels = ['unknown', 'what', 'when', 'who', 'affirmation']
prediction_labels=[]
for p in predictions:
    prediction_labels.append(labels[np.argmax(p)])

148/148 [==============================] - 116s 781ms/step


In [34]:
prediction_labels

['who',
 'what',
 'what',
 'what',
 'who',
 'what',
 'affirmation',
 'unknown',
 'what',
 'what',
 'unknown',
 'what',
 'what',
 'what',
 'what',
 'what',
 'who',
 'who',
 'who',
 'who',
 'what',
 'who',
 'what',
 'affirmation',
 'unknown',
 'who',
 'what',
 'unknown',
 'what',
 'what',
 'what',
 'who',
 'what',
 'what',
 'what',
 'what',
 'who',
 'unknown',
 'who',
 'who',
 'what',
 'what',
 'what',
 'what',
 'unknown',
 'unknown',
 'what',
 'unknown',
 'affirmation',
 'unknown',
 'who',
 'unknown',
 'what',
 'unknown',
 'what',
 'affirmation',
 'unknown',
 'what',
 'who',
 'what',
 'unknown',
 'who',
 'who',
 'what',
 'what',
 'what',
 'when',
 'what',
 'unknown',
 'unknown',
 'what',
 'what',
 'unknown',
 'what',
 'what',
 'what',
 'who',
 'what',
 'what',
 'when',
 'what',
 'what',
 'who',
 'unknown',
 'what',
 'who',
 'unknown',
 'what',
 'who',
 'unknown',
 'who',
 'unknown',
 'what',
 'what',
 'what',
 'unknown',
 'who',
 'what',
 'what',
 'who',
 'what',
 'unknown',
 'what',
 '

In [ ]:
        +
    
    